# Run Python file

In [ ]:
from GraphDB import GraphDB

graphdb = GraphDB(json_file = "../ba_data_extraction/banking_act.json",
                      visualisation = True)
graphdb.run()

# Connect to Neo4j

In [ ]:
import dotenv
import os
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from py2neo import Graph, Node, Relationship
import json
import networkx as nx
import matplotlib.pyplot as plt

load_status = dotenv.load_dotenv("../Neo4j-66cb9e32-Created-2025-03-26.txt")
if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

# Load json to graph

In [ ]:
def connect_to_database(): 
    driver = GraphDatabase.driver(URI, auth=AUTH)
    driver.verify_connectivity()
    print("Connection established.")
    return driver

def load_json_to_graph(json_file):
    driver = connect_to_database()

    with open(json_file, "r") as file:
        data = json.load(file)

    #sections = data.get("sections", [])

    with driver.session() as session:
        # Create nodes
        for data_section in data:
            text_id = data_section.get("id")
            text = data_section.get("text", "")
            metadata = data_section.get("metadata")
            references = metadata.get('references', []) if metadata else []
            
            session.run("""
                MERGE (s:Section {id: $text_id})
                SET s.text_id = $text_id, s.text = $text
            """, text_id=text_id, text=text) 

            # Create downstream relationships (current section → references)
            for ref_id in references:
                session.run("""
                    MATCH (s1:Section {id: $text_id})
                    MATCH (s2:Section {id: $ref_id})
                    MERGE (s1)-[:REFERS_TO]->(s2)
                """, text_id=text_id, ref_id=ref_id)

                # Create upstream relationships (references → current section)
                session.run("""
                    MATCH (s1:Section {id: $ref_id})
                    MATCH (s2:Section {id: $text_id})
                    MERGE (s1)-[:REFERRED_BY]->(s2)
                """, text_id=text_id, ref_id=ref_id)

    driver.close()

In [ ]:
json_file_path = "../ba_data_extraction/banking_act.json"
load_json_to_graph(json_file_path)


## Visualisation 

In [ ]:
def fetch_graph_data():
    driver = GraphDatabase.driver(URI, auth=AUTH)
    G = nx.DiGraph()

    with driver.session() as session:
        # Fetch nodes
        nodes = session.run("MATCH (n:Section) RETURN n.id")
        for record in nodes:
            G.add_node(record["n.id"])

        # Fetch edges
        edges = session.run("""
            MATCH (a:Section)-[r:REFERS_TO]->(b:Section)
            RETURN a.id AS source, b.id AS target
        """)
        for record in edges:
            G.add_edge(record["source"], record["target"])

    driver.close()
    return G

In [ ]:
def fetch_sample_graph_data():
    driver = GraphDatabase.driver(URI, auth=AUTH)
    G = nx.DiGraph()

    with driver.session() as session:
        # Fetch nodes
        nodes = session.run("MATCH (n:Section) RETURN n.id")
        for record in nodes:
            G.add_node(record["n.id"])

        # Fetch edges
        edges = session.run('''
            MATCH (a:Section)-[r]->(b:Section)  
            RETURN a, r, b  
            LIMIT 10;
            ''')
        
    driver.close()
    return G

In [ ]:
G = fetch_sample_graph_data()
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, k=0.5)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color="lightblue", edge_color="gray", font_size=10, font_weight="bold")
plt.show()

In [ ]:
G = fetch_graph_data()
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, k=0.5)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color="lightblue", edge_color="gray", font_size=10, font_weight="bold")
plt.show()

In [ ]:
query = "MATCH (n)-[r]->(m) RETURN n, r, m"
results = graph.run(query)

# Create NetworkX graph
G = nx.DiGraph()

# Add nodes and edges
for record in results:
    node1 = record["n"]["name"]
    node2 = record["m"]["name"]
    relationship = record["r"].type
    
    G.add_node(node1)
    G.add_node(node2)
    G.add_edge(node1, node2, label=relationship)

# Draw the graph
plt.figure(figsize=(8, 5))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray", node_size=3000, font_size=10)
edge_labels = {(u, v): d["label"] for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
plt.title("Neo4j Graph Visualization")
plt.show()


In [ ]:
for heading, content in zip(headings_level_1, content_level_1):
    section_node = Node("Section", name=heading, content=content)
    graph.create(section_node)
    
    for subheading, subcontent in zip(headings_level_2, content_level_2):
        if subheading.startswith(heading):  # Ensuring it's a child of the current section
            subsection_node = Node("Subsection", name=subheading, content=subcontent)
            graph.create(subsection_node)
            rel = Relationship(section_node, "HAS_SUBSECTION", subsection_node)
            graph.create(rel)


In [ ]:
def count_connected_components():
    query = """
    CALL gds.wcc.stream('myGraph')
    YIELD nodeId, componentId
    RETURN COUNT(DISTINCT componentId) AS num_connected_components;
    """
    
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            result = session.run(query)
            for record in result:
                print("Number of Connected Components:", record["num_connected_components"])

    driver.close()

In [ ]:
count_connected_components()